In [2]:
%load_ext autoreload
%autoreload 2

from dataclasses import dataclass
from typing import List, Literal

from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env file

from openai import OpenAI
client = OpenAI()

import pandas as pd



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Define target internal schemas

BlockType = Literal["text", "python"]
SectionID = Literal["concept", "example", "exercise"]


@dataclass
class BlockDraft:
    type: BlockType
    content: str


@dataclass
class SectionDraft:
    id: SectionID
    title: str
    minutes: int
    blocks: List[BlockDraft]


@dataclass
class LessonDraft:
    objective: str
    sections: List[SectionDraft]

In [4]:
#Human readable schema for prompt mathcing previous cell data classes

EXPECTED_SCHEMA = """
{
  "objective": "Describe the learner outcome in one sentence.",
  "sections": [
    {
      "id": "concept",
      "title": "Key idea or principle",
      "minutes": 5,
      "blocks": [
        {
          "type": "text",
          "content": "Explain the concept in plain language."
        }
      ]
    },
    {
      "id": "example",
      "title": "Worked example",
      "minutes": 5,
      "blocks": [
        {
          "type": "text",
          "content": "Walk through the example step by step."
        },
        {
          "type": "python",
          "content": "print('Example code here')"
        }
      ]
    },
    {
      "id": "exercise",
      "title": "Practice task",
      "minutes": 5,
      "blocks": [
        {
          "type": "text",
          "content": "Provide an exercise prompt."
        }
      ]
    }
  ]
}
"""
print(EXPECTED_SCHEMA)


{
  "objective": "Describe the learner outcome in one sentence.",
  "sections": [
    {
      "id": "concept",
      "title": "Key idea or principle",
      "minutes": 5,
      "blocks": [
        {
          "type": "text",
          "content": "Explain the concept in plain language."
        }
      ]
    },
    {
      "id": "example",
      "title": "Worked example",
      "minutes": 5,
      "blocks": [
        {
          "type": "text",
          "content": "Walk through the example step by step."
        },
        {
          "type": "python",
          "content": "print('Example code here')"
        }
      ]
    },
    {
      "id": "exercise",
      "title": "Practice task",
      "minutes": 5,
      "blocks": [
        {
          "type": "text",
          "content": "Provide an exercise prompt."
        }
      ]
    }
  ]
}



In [5]:
PROMPT_TEMPLATE = """
You are an expert instructor. Create a concise 15-minute lesson on "{topic}" for a "{level}" learner.

Return VALID JSON ONLY that matches this schema (no extra keys):
{schema}

Hard requirements:
- Total minutes across sections must sum to exactly 15.
- Use 1-3 sections total.
- section.id must be one of: "concept", "example", "exercise".
- Each section must include at least 1 block.
- block.type must be "text" or "python".
- At least one block.type must be "python".
- Python blocks must be runnable, minimal, and directly related to the topic.

Quality guidelines:
- Objective is one sentence, specific and measurable.
- concept: teach the key idea(s) in plain language.
- example: walk through a small, concrete example with code and short explanations.
- exercise: give a short practice task; no solutions.

Formatting rules:
- Output JSON only, no markdown fences, no commentary.
- Use double quotes for all keys/strings.

Now produce the JSON.
"""

print(PROMPT_TEMPLATE.format(
    topic="pandas groupby performance",
    level="beginner",
    schema=EXPECTED_SCHEMA
))



You are an expert instructor. Create a concise 15-minute lesson on "pandas groupby performance" for a "beginner" learner.

Return VALID JSON ONLY that matches this schema (no extra keys):

{
  "objective": "Describe the learner outcome in one sentence.",
  "sections": [
    {
      "id": "concept",
      "title": "Key idea or principle",
      "minutes": 5,
      "blocks": [
        {
          "type": "text",
          "content": "Explain the concept in plain language."
        }
      ]
    },
    {
      "id": "example",
      "title": "Worked example",
      "minutes": 5,
      "blocks": [
        {
          "type": "text",
          "content": "Walk through the example step by step."
        },
        {
          "type": "python",
          "content": "print('Example code here')"
        }
      ]
    },
    {
      "id": "exercise",
      "title": "Practice task",
      "minutes": 5,
      "blocks": [
        {
          "type": "text",
          "content": "Provide an exercis

In [6]:
# Make a call to openAI with the above prompt 

response = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that creates lesson plans in JSON format."
        },
        {
            "role": "user",
            "content": PROMPT_TEMPLATE.format(
                topic="pandas groupby performance",
                level="beginner",
                schema=EXPECTED_SCHEMA
            )
        }
    ],
    temperature=0.7,
    max_tokens=1500,
)
lesson_json = response.choices[0].message.content
print(lesson_json)

{
  "objective": "Understand how to use pandas groupby efficiently and recognize factors affecting its performance.",
  "sections": [
    {
      "id": "concept",
      "title": "Understanding pandas groupby and its performance factors",
      "minutes": 5,
      "blocks": [
        {
          "type": "text",
          "content": "The pandas groupby function splits data into groups based on column values and applies aggregation or transformation operations. While groupby is powerful, performance depends on factors like data size, number of groups, and the complexity of the aggregation function. Using built-in aggregation methods and minimizing custom functions improves speed."
        }
      ]
    },
    {
      "id": "example",
      "title": "Grouping and aggregating data with pandas",
      "minutes": 5,
      "blocks": [
        {
          "type": "text",
          "content": "We will group a small DataFrame by a column and calculate the sum of another column. This uses a built-

In [7]:
import json
json.loads(lesson_json)


{'objective': 'Understand how to use pandas groupby efficiently and recognize factors affecting its performance.',
 'sections': [{'id': 'concept',
   'title': 'Understanding pandas groupby and its performance factors',
   'minutes': 5,
   'blocks': [{'type': 'text',
     'content': 'The pandas groupby function splits data into groups based on column values and applies aggregation or transformation operations. While groupby is powerful, performance depends on factors like data size, number of groups, and the complexity of the aggregation function. Using built-in aggregation methods and minimizing custom functions improves speed.'}]},
  {'id': 'example',
   'title': 'Grouping and aggregating data with pandas',
   'minutes': 5,
   'blocks': [{'type': 'text',
     'content': 'We will group a small DataFrame by a column and calculate the sum of another column. This uses a built-in aggregation function, which is efficient.'},
    {'type': 'python',
     'content': "import pandas as pd\n\ndat

In [8]:
# Batch testing with different topics and levels
TEST_CASES = {
    "programming_beginner": {
        "topic": "pandas groupby performance",
        "level": "beginner",
    },
    "programming_intermediate": {
        "topic": "optimize pandas groupby memory usage",
        "level": "intermediate",
    },
    "data_concept_beginner": {
        "topic": "correlation vs causation",
        "level": "beginner",
    },
    "data_concept_intermediate": {
        "topic": "bias vs variance trade-off",
        "level": "intermediate",
    },
    "analytics_reasoning_beginner": {
        "topic": "interpreting A/B test results",
        "level": "beginner",
    },
    "analytics_reasoning_intermediate": {
        "topic": "common pitfalls in KPI interpretation",
        "level": "intermediate",
    },
    "statistics_intuition_beginner": {
        "topic": "what a p-value means",
        "level": "beginner",
    },
}

In [9]:
def generate_lesson(topic: str, level: str, temperature: float = 0.7, max_tokens: int = 1500) -> dict:
    """
    Generate a lesson plan using OpenAI API.
    
    Args:
        topic: The lesson topic
        level: The learner level (e.g., "beginner", "intermediate")
        temperature: OpenAI temperature parameter (default: 0.7)
        max_tokens: Maximum tokens in response (default: 1500)
    
    Returns:
        dict with keys: topic, level, valid_json, lesson_data, raw_output
    """
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant that creates lesson plans in JSON format."
            },
            {
                "role": "user",
                "content": PROMPT_TEMPLATE.format(
                    topic=topic,
                    level=level,
                    schema=EXPECTED_SCHEMA
                )
            }
        ],
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content


In [10]:
# Run tests and collect results in a dataframe
results = []
for case_name, case_params in TEST_CASES.items():
    print(f"Generating lesson for case: {case_name}")
    raw_output = generate_lesson(
        topic=case_params["topic"],
        level=case_params["level"]
    )
    try:
        lesson_data = json.loads(raw_output)
        valid_json = True
    except json.JSONDecodeError:
        lesson_data = None
        valid_json = False

    results.append({
        "case": case_name,
        "topic": case_params["topic"],
        "level": case_params["level"],
        "valid_json": valid_json,
        "lesson_data": lesson_data,
        "raw_output": raw_output,
    })
results_df = pd.DataFrame(results)

Generating lesson for case: programming_beginner
Generating lesson for case: programming_intermediate
Generating lesson for case: data_concept_beginner
Generating lesson for case: data_concept_intermediate
Generating lesson for case: analytics_reasoning_beginner
Generating lesson for case: analytics_reasoning_intermediate
Generating lesson for case: statistics_intuition_beginner


In [11]:
results_df

,case,topic,level,valid_json,lesson_data,raw_output
0,programming_beginner,pandas groupby performance,beginner,True,{'objective': 'Understand how to use pandas gr...,"{\n ""objective"": ""Understand how to use panda..."
1,programming_intermediate,optimize pandas groupby memory usage,intermediate,True,{'objective': 'Learn to reduce memory usage wh...,"{\n ""objective"": ""Learn to reduce memory usag..."
2,data_concept_beginner,correlation vs causation,beginner,True,{'objective': 'Explain the difference between ...,"{\n ""objective"": ""Explain the difference betw..."
3,data_concept_intermediate,bias vs variance trade-off,intermediate,True,{'objective': 'Explain the bias-variance trade...,"{\n ""objective"": ""Explain the bias-variance t..."
4,analytics_reasoning_beginner,interpreting A/B test results,beginner,True,{'objective': 'Explain how to interpret the re...,"{\n ""objective"": ""Explain how to interpret th..."
5,analytics_reasoning_intermediate,common pitfalls in KPI interpretation,intermediate,True,{'objective': 'Identify and avoid common pitfa...,"{\n ""objective"": ""Identify and avoid common p..."
6,statistics_intuition_beginner,what a p-value means,beginner,True,{'objective': 'Explain what a p-value represen...,"{\n ""objective"": ""Explain what a p-value repr..."


In [15]:
# save results to a csv file in the data folder
results_df.to_csv("data/lesson_generation_test_results.csv", index=False)